# Задание 6

### Пример входных данных

In [1]:
import pandas as pd
clinton = pd.read_csv("clinton.csv", encoding="utf-8")
trump = pd.read_csv("trump.csv", encoding="utf-8")

In [2]:
trump['text'][0]

'Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc'

In [3]:
str(trump.loc[0][0])

'Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc'

In [4]:
clinton.head()

,text
0,The question in this election: Who can put the...
1,"Last night, Donald Trump said not paying taxes..."
2,Couldn't be more proud of @HillaryClinton. Her...
3,"If we stand together, there's nothing we can't..."
4,Both candidates were asked about how they'd co...


## Описание задания

Предлагается написать генератор случайных диалогов на основе твитов Трампа (файл <i>"trump.csv"</i>) и Хеллори Клинтон (файл <i>"clinton.csv"</i>).

* Каждый диалог состоит из нескольких "ходов" (<i>turn</i>).
* Каждый такой <i>turn</i> представляет собой цепочку нескольких реплик разных агентов (участников диалога, их может быть больше двух).
* Цепочка реплик представляет собой набор реплик агентов. Первое сообщение в цепочке - <i>"исходное сообщение"</i> на которое отвечают оставшиеся агенты (по одному сообщению за turn).

Программа должна представлять отдельный пакет следующей структуры:
<ul>
<li>run\_generator.py</li>
<li>dialogsgenerator:
<ul>
<li>agent.py</li>
<li>randomdialog.py</li>
<li>\_\_init__.py</li>
</ul></li>
</ul>

<b>URGENT:</b> в работе ЗАПРЕЩАЕТСЯ использовать циклы:
<ul>
<li>Использование 1 for (или while): max 1 балл</li>
<li>Использование до 4 for (или while): max 0.8 баллов</li>
<li>Использование 4+ for (или while): max 0.5 баллов</li>
</ul>

### Файл agent.py

Описание класса агента (один класс для всех агентов, в нашем случае для Трампа и Клинтон).

In [5]:
from random import randint

In [511]:
from collections import Generator


class Agent(Generator):
    
    def __init__(self, kb, name):
        # Инициализация
        self.tweets_pool = kb['text']
        self.num_tweets = len(kb)
#         print("i'm in init")
        self.name = name

    def send(self, msg):
        # Будем возвращать случайное сообщение из self.tweets_pool
        random_idx = randint(0, self.num_tweets - 1)
#         print("agent name is ", self.name)
        return "{} : {}".format(self.name, self.tweets_pool[random_idx])

    def throw(self, typ=None, val=None, tb=None):
        # Оставляем как есть
        super().throw(typ, val, tb)

    def __str__(self):
        # Опишем строковое представление класса
        return self.name

    def __repr__(self):
        return str(self)

#### Пример использования:

In [512]:
# from dialogsgenerator import Agent
clint, trum = Agent(clinton, "clinton"), Agent(trump, "trump")
clint.send("hello")
# type(clint)
clint.send("bla bla")

'clinton : Donald Trump has built his campaign on prejudice and paranoia. He’s taking hate groups mainstream.'

In [513]:
# print(next(clint))
# print(next(clint))
# print(next(clint))
# print(next(clint))

### Файл randomdialog.py

Описание класса, генерирующего случайный диалог.

In [429]:
import sys
from itertools import chain, repeat, cycle, islice, takewhile
from collections import Counter

In [442]:
class RandomDialog(object):
    
    def __init__(self, agents, max_len=5):
        # Инициализация
        self.agents = agents
        self.agents_num = len(agents)
        self.max_len = max_len
    
    def generate(self):
        """
        Генерирует случайный диалог состоящий из 1-max_len цепочек.
        При генерации вызывает функцию turn.
        Возвращаемый объект является генератором.
        """
        
        """
        ______________________________________________________________________________________
            Мы хотим вернуть (max_len - 1) цепочек ходов, для этого нам нужно (max_len - 1) раз 
            вызвать turn(), но чтобы каждый раз метод turn() возвращал разные диалоги.

             * self_n_times - пары (i, self), где i = 0..max_len-1
             * invoke_turn - вызывает функцию turn у своего второго аргумента (второй аргумент - self)

            Возьмём map от self_n_times, к котором применён invoke_turn, получим то, что нужно            
        """
        
        # пары - (i, self), где i = 1..max_len-1
        self_n_times = zip(range(self.max_len - 1), repeat(self))
        
        # вызывает turn() у второго аргумента
        invoke_turn = lambda args: list(args[1].turn())
        
        # для каждой пары (i, self) вызываем invoke_turn от self
        yield from map(invoke_turn, self_n_times)

    def turn(self):
        """
        Генерирует одну случайную цепочку следующим образом: выбирается случайный агент.
        Он "говорит" случайное сообщение (msg) из своей Agent.kb (используйте next или send(None)).
        (правила того, как выбирать никак не регулируются, вы можете выбирать случайный твит из Agent.kb никак не учитывая
        переданное msg).
        Это сообщение передается с помощью send (помним, что агент - это объект-генератор).
        Далее получаем ответ от всех агентов и возвращаем (генерируем) их (включая исходное сообщение).
        Возвращаемый объект является генератором.
        """
        
        # получаем сообщение от случайного агента
        rand_agent_ind = randint(0, self.agents_num - 1)
        msg = next(self.agents[rand_agent_ind])
        
        # отправляем сообщение всем агентам
        _ = list(map(lambda x: x.send(msg), self.agents))

        # возвращаем ответы всех агентов вместе с исходным сообщением
        it = iter(self.agents)
        yield from chain([msg], list(map(next, it)))

    def eval(self, dialog=None):
        """
        Превращает генератор случайного далога (который возвращается в self.generate())
        в список списков (пример использования далее).
        Возвращаемый объект является списком.
        """
        if dialog is None:
            dialog = self.generate()
                
        return list(dialog)      

    def write(self, dialog=None, target=sys.stdout):
        """
        Записывает результат self.eval() в соответствующий поток.
        """
        
        if dialog is None:
            dialog = self.eval()
        
        sep_turns = lambda turn: "\t" + turn
        
        turn_desc = lambda turn_obj: list(chain(["turn {}:".format(turn_obj[0])], 
                                           map(sep_turns, turn_obj[1])))
        
        print_dialogs = lambda turn: list(map(lambda x: print(x, file=target), turn_desc(turn)))
        
        list(map(print_dialogs, enumerate(dialog)))
        
    pass


In [443]:
test_random_diaog = RandomDialog([Agent(clinton, "clinton"), Agent(trump, "trump")], max_len=3)

In [444]:
gen = test_random_diaog.turn()
list(gen)
# list(next(gen))

["clinton ---> It's a simple idea: Those who have benefited the most from our economy should pay their fair share in taxes. https://t.co/2nSFZtgYFJ",
 'clinton ---> "I know how hard this job is. You need steadiness as well as strength and smarts... Donald Trump is not qualified." —Hillary on @CNN',
 'trump ---> Wow, every poll said I won the debate last night. Great honor!']

In [445]:
gen_dialog = test_random_diaog.generate()
list(test_random_diaog.generate())

[["clinton ---> We know all too well who Donald Trump is. Let's make sure he never becomes president: https://t.co/tTgeqxNqYm https://t.co/3lJjQEQ0BR",
  "clinton ---> .@realDonaldTrump says he’ll be “the best for women.”\n\nDon't think so. https://t.co/zsXgf8wQF2",
  'trump ---> In Iran deal we get 4 prisoners. They get $150 billion, 7 most wanted and many off watch list. This will create great incentive for others!'],
 ['clinton ---> "It\'s pretty obvious he doesn\'t know a lot about the issues." —@SenateMajLdr on Trump\n\nWe couldn\'t agree more, Mitch https://t.co/YO81aW6XZB',
  'clinton ---> Last night, Donald Trump said not paying taxes was "smart." You know what I call it? Unpatriotic. https://t.co/t0xmBfj7zF',
  'trump ---> Low energy candidate @JebBush has wasted $80 million on his failed presidential campaign. Millions spent on me. He should go home and relax!']]

In [446]:
evaluate = test_random_diaog.eval(gen_dialog)
# test_random_diaog.eval(gen_dialog)

In [447]:
test_random_diaog.write(evaluate)

----------- turn 0: -------------
	clinton ---> When we come together, there's no problem we can't solve and no divide we can't heal. https://t.co/2eutCA4R6C
	clinton ---> We’re going to invest in union training programs and apprenticeship programs so you can earn while you learn: https://t.co/sbbiIqIz0c
	trump ---> Got the endorsement of Brian France and @NASCAR yesterday in Georgia. Also, many of the sports great drivers. Thank you Nascar and Georgia!
----------- turn 1: -------------
	trump ---> I heard that the underachieving John King of @CNN on Inside Politics was one hour of lies. Happily, few people are watching - dead network!
	clinton ---> We are living off of the infrastructure built by our parents and grandparents. It’s overdue for upgrades: https://t.co/KjhFN0QS2c
	trump ---> Thank you- on my way!  https://t.co/uTCXPbWpve


#### Пример использования:

In [448]:
# from dialogsgenerator import RandomDialog
rd = RandomDialog([Agent(clinton, "clinton"), Agent(trump, "trump")], max_len=2)

In [855]:
generated = rd.generate()
generated

<generator object RandomDialog.generate at 0x0000007DC85901A8>

In [856]:
evaled = rd.eval(generated)
evaled

[["clinton: Let's make it easier for young people to start businesses and fulfill their dreams. https://t.co/cJzNCnaahh https://t.co/vZHW9VZKDT",
  'trump: My list of potential U.S. Supreme Court Justices was very well recieved. During the next number of weeks I may be adding to the list!'],
 ["trump: DON'T LET HILLARY CLINTON DO IT AGAIN!\r #TrumpPence16\r https://t.co/1mGkPNZPKF",
  'clinton: With the first local transmission of Zika in FL, it’s even more critical for Republicans to stop blocking action: https://t.co/KjmWMDOgDl']]

In [857]:
rd.write(evaled)


turn 0:
	clinton: Let's make it easier for young people to start businesses and fulfill their dreams. https://t.co/cJzNCnaahh https://t.co/vZHW9VZKDT
	trump: My list of potential U.S. Supreme Court Justices was very well recieved. During the next number of weeks I may be adding to the list!
turn 1:
 https://t.co/1mGkPNZPKF
	clinton: With the first local transmission of Zika in FL, it’s even more critical for Republicans to stop blocking action: https://t.co/KjmWMDOgDl

### Файл run_generator.py

Содержит функции для генерации и записи нескольких диалогов. Файл должен быть написан так, чтобы его можно было запускать через командную строку:
<img src="cmd.png">
Описание аргументов представлено на рисунке:
<img src="cmd_help.png">
Для разбора аргументов ипользуйте <a href="https://docs.python.org/3/howto/argparse.html">модуль argparse</a>. Все представленные на рисунке аргументы должны быть обработаны (кроме help, он обрабатывается автоматически модулем argparse).

In [501]:
from functools import reduce
from colorama import Style

In [508]:
def generate(rd, count_dialogs=5):
    """
    Генерирует count_dialogs диалогов с помощью rd.generate().
    Возвращаемый объект является генератором.
    """
    # пары - (i, rd), где i = 1..count_dialogs
    rd_n_times = zip(range(count_dialogs), repeat(rd))

    # вызывает generate() у второго аргумента
    invoke_generate = lambda args: list(args[1].generate())

    # у каждой пары (i, rd) вызываем invoke_generate от rd
    yield from map(invoke_generate, rd_n_times)
    

def write(dialogs, target):
    """
    Записывает сгенерированные диалоги dialogs (это объект-генератор) в поток target.
    """
    sep_turns = lambda turn: "\t" + turn
        
    turn_desc = lambda turn_obj: list(chain(["turn {}:".format(turn_obj[0])], map(sep_turns, turn_obj[1])))
    
    concat_turns = lambda turn_obj: reduce(lambda x, y: x + y, map(turn_desc, turn_obj))
    
    dialog_desc = lambda dialog_obj: list(chain(["_________________Dialog {}_________________".format(dialog_obj[0])], 
                                    concat_turns(enumerate(dialog_obj[1]))))
                                           
    print_dialogs = lambda dialog_obj: list(map(lambda x: print(x, file=target), dialog_desc(dialog_obj)))
        
    list(map(print_dialogs, enumerate(dialogs)))


if __name__ == "__main__":
    print("sdf")


sdf


In [509]:
dialogs = generate(test_random_diaog, 2)
# list(dialogs)

In [510]:
write(dialogs, sys.stdout)

_________________Dialog 0_________________
turn 0:
	trump ---> Russia took Crimea during the so-called Obama years. Who wouldn't know this and why does Obama get a free pass?
	clinton ---> Let's stand together to do all the good we can, in all the ways we can, for as long as we can.
https://t.co/jHaoZKXURE
	trump ---> I will be live tweeting @megynkelly Show in 10 minutes. Should be interesting. Will be on Fox Network! ENJOY!
turn 1:
	clinton ---> Cheryl lost her husband in Iraq. Then Trump's company targeted her and scammed her out of $35,000. https://t.co/Wjz6zohMIT
	clinton ---> Happy National Dog Day! https://t.co/qmpQsNQxGL
	trump ---> Thank you America! #Trump2016 https://t.co/KL7VZbM1jP
_________________Dialog 1_________________
turn 0:
	trump ---> "@vikkideiter: Something VERY close to my heart. I'm a NAVY VET! I love @realDonaldTrump's  VETERANS ADMINISTRATION REFORMS.
	clinton ---> Too many veterans and their families aren’t getting the mental health services they need. Our p

#### Пример использования функций из файла:

In [869]:
import sys
from run_generator import generate, write

In [870]:
dialogs = generate(rd, 2)
dialogs

In [871]:
write(dialogs, sys.stdout)

________________________________________Dialog 0________________________________________
turn 0:
	trump: Crooked Hillary wants to take your 2nd Amendment rights away. Will guns be taken from her heavily armed Secret Service detail? Maybe not!
	clinton: "Bill, that conversation we started in the law library 45 years ago, it is still going strong.” —Hillary
turn 1:
	clinton: Our teachers deserve more than just a pat on the back. They deserve a raise. #TeacherAppreciationDay https://t.co/0sgmUFg6mX
	trump: I visited our Trump Tower campaign headquarters last night, after returning from Ohio and Arizona, and it was packed with great pros - WIN!
________________________________________Dialog 1________________________________________
turn 0:
	clinton: "He won me over with that Mexican rapist speech." —Ann Coulter on Donald Trump https://t.co/GTNOhKYMA2
 #MakeAmericaGreatAgain #Trump2016 https://t.co/7WHS3vbXSw
turn 1:
	trump: .@FreeJesseJames- Just read your complete statement. You are an am